<a href="https://colab.research.google.com/github/Samenergy/Formative-1-Databases/blob/main/Peer_group_13_Formative_2_Data_Preprocessing__Jules_Gatete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Formative 2- Data Preprocessing Assignment for Machine Learning Pipeline**

## Group Members

1.   Juliana Crystal Holder
2.   Jules Gatete
3.  Samuel Dushime


# Assignment Overview
In this assignment, we'll be working with real-world datasets that contain overlapping but different features. The goal is to augment, merge, and enhance the data while ensuring consistency in a machine learning pipeline.
The assignment is divided into three main parts:


*   Data Augmentation on CSV Files
*   Merging Datasets with Transitive Properties
*   Data Consistency and Quality Checks

This notebook contains the implementation for all three parts, along with detailed explanations and visualizations of the approach taken at each step.



**Part 2: Data Augmentation on CSV Files (Jules Gatete)**

In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import random

In [15]:
transactions_df = pd.read_csv("customer_transactions_augmented.csv")
social_profiles_df = pd.read_csv("customer_social_profiles.csv")
id_mapping_df = pd.read_csv("id_mapping.csv")

In [16]:
# Merge transactions with ID mapping
merged_df = transactions_df.merge(id_mapping_df, on="customer_id_legacy", how="left")

<ipython-input-16-987c7a555462>:2: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  merged_df = transactions_df.merge(id_mapping_df, on="customer_id_legacy", how="left")


In [17]:
# Merge with social profiles
final_df = merged_df.merge(social_profiles_df, on="customer_id_new", how="left")

In [18]:
# Handle missing values
final_df.fillna({"engagement_score": final_df["engagement_score"].median(),
                 "purchase_interest_score": final_df["purchase_interest_score"].median(),
                 "review_sentiment": "Neutral"}, inplace=True)

In [19]:
# Aggregate transaction data per customer
aggregated_df = final_df.groupby("customer_id_new").agg({
    "purchase_amount": ["sum", "mean"],
    "customer_rating": "mean",
    "engagement_score": "mean",
    "purchase_interest_score": "mean"
}).reset_index()


In [20]:
# Rename columns
aggregated_df.columns = ["customer_id_new", "total_spent", "avg_spent", "avg_rating",
                         "avg_engagement_score", "avg_purchase_interest_score"]

In [22]:
# Compute Customer Engagement Score
aggregated_df["customer_engagement_score"] = (
    0.5 * aggregated_df["avg_engagement_score"] +
    0.3 * aggregated_df["avg_purchase_interest_score"] +
    0.2 * aggregated_df["avg_rating"]
)

In [24]:
# TF-IDF on customer reviews (if available)
from sklearn.feature_extraction.text import TfidfVectorizer # Import the necessary class

if "review_sentiment" in final_df.columns:
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(final_df["review_sentiment"].astype(str))
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    final_df = pd.concat([final_df, tfidf_df], axis=1)

In [25]:
# Save final dataset
output_file = "final_customer_data_13.csv"
aggregated_df.to_csv(output_file, index=False)

print(f"Final processed dataset saved as {output_file}")

Final processed dataset saved as final_customer_data_13.csv
